# Table of Contents

[Imports](#Imports)

[Playground](#Playground)

[Spotify Functions](#Spotify-Functions)

[Songkick Functions](#Songkick-Functions)

[Analysis Functions](#Analysis-Functions)

[Analysis](#Analysis)

# Imports

[Back to top](#Table-of-Contents)

In [2]:
#import necessary libraries
import requests
import json
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import re
import spotipy
import base64
import teetool as tt
from collections import Counter
import libpysal
from pysal.model import spreg
from libpysal.weights import Queen, Rook, KNN, Kernel
import pysal
from pysal.explore import esda
from pysal.viz.splot import esda as esdaplot
import cenpy
# import movingpandas as mpd
import datetime
from datetime import date
from spotipy.oauth2 import SpotifyClientCredentials
from census import Census
import censusgeocode
import us
import folium
import statsmodels.formula.api as sm
import numpy
import statistics
import itertools as it
import math

api_key = "fCnCRQ1S6LDAS7zQ"
client_id = 'f5a0fe76389c42c0b85c832f1de66cf7'
client_secret = 'e9d4fc0874f345b2b636a6e307384bb7'
census_key = '684a12c6b65d2fe14bdf6d8354da8fe3aff08d70'
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

cen = Census(census_key)


world = gpd.read_file('Countries_WGS84/Countries_WGS84.shp') #open file
states_shp = gpd.read_file('states_21basic/states.shp') #open file

s2 = states_shp[['geometry', 'STATE_ABBR']]
s2.columns = ['geometry', 'name']

w2 = world[['geometry', 'CNTRY_NAME']]
w2.columns = ['geometry', 'name']

ws = w2.append(s2)

print(ws)


#maybe remove festivals - songkick
# maybe market - spotify

/Users/kalebtsegaye/miniconda3/envs/env1/lib/python3.7/site-packages/pysal/explore/segregation/network/network.py:16: UserWarning: You need pandana and urbanaccess to work with segregation's network module
You can install them with  `pip install urbanaccess pandana` or `conda install -c udst pandana urbanaccess`
  "You need pandana and urbanaccess to work with segregation's network module\n"
/Users/kalebtsegaye/miniconda3/envs/env1/lib/python3.7/site-packages/pysal/model/spvcm/abstracts.py:10: UserWarning: The `dill` module is required to use the sqlite backend fully.
  from .sqlite import head_to_sql, start_sql


                                             geometry                 name
0   POLYGON ((-69.8822326660156 12.4111099243165, ...                Aruba
1   (POLYGON ((-61.7388916015625 17.5405540466309,...  Antigua and Barbuda
2   POLYGON ((61.2765579223633 35.6072463989258, 6...          Afghanistan
3   POLYGON ((-5.15213489532459 30.1804695129396, ...              Algeria
4   (POLYGON ((45.0258293151855 41.0305480957031, ...           Azerbaijan
..                                                ...                  ...
46  POLYGON ((-94.46169128436209 34.19676535931296...                   AR
47  (POLYGON ((-93.70752428365972 30.2395783556275...                   LA
48  (POLYGON ((-80.7856622716253 28.78519435427305...                   FL
49  (POLYGON ((-88.49752727880752 48.1737953723300...                   MI
50  (POLYGON ((-161.3337853466416 58.7332483821643...                   AK

[302 rows x 2 columns]


# Playground

[Back to top](#Table-of-Contents)

# Spotify Functions

[Back to top](#Table-of-Contents)

In [3]:
# spotify function definitions

# given a name/id or genre, will provide a list of 
# artists that exist in relation and also are within the follower limit

# custom function used to sort by num of shared genres
def custom_sort(x):
    return x[2]

query_count = 0

# handles counting the genres for related artists
def ra_help(artid, genres, followers, lessthan):
    global query_count
#     print(followers)
    alist = []
    if query_count < 10:
        query_count += 1
        for a in sp.artist_related_artists(artid)['artists']:
#             print(a)
            gc = 0
            if (followers == -1):
                for g in a['genres']:    
                    if g in genres:
                        gc += 1
                if gc > 0 and ([item for item in alist if item[1] == a['id']] == []):
                    alist.append((a['name'], a['id'], gc, a['followers']['total']))
            else:
                if (lessthan == True):
#                     print(a)
                    if (a['followers']['total'] < followers):
                        for g in a['genres']:    
                            if g in genres:
                                gc += 1
                        if gc > 0 and ([item for item in alist if item[1] == a['id']] == []):
                            alist.append((a['name'], a['id'], gc, a['followers']['total']))
                elif (lessthan == False):
                    if (a['followers']['total'] >= followers):
                        for g in a['genres']:    
                            if g in genres:
                                gc += 1
                        if gc > 0 and ([item for item in alist if item[1] == a['id']] == []):
                            alist.append((a['name'], a['id'], gc, a['followers']['total']))
                
        return sorted(alist, key = custom_sort, reverse = True)
                    
# used to faciliate recursive calls - 
# allows me to find the related artists in each level of separation from the root
# before moving on to the root artist
def ra_rec(lst, genres, followers, lessthan):
    global query_count
    lst2 = []
    query_count += 1
    for a in lst:
        if query_count < 10:
            lst2+=(ra_help(a[1], genres, followers, lessthan))
#         print("ra_rec ra_help" + str(query_count))
    if query_count < 10:
        lst2 += (ra_rec((sorted(lst2, key = custom_sort)), genres, followers, lessthan))    
    
    return sorted(lst2, key = custom_sort, reverse = True)

def related_artists(artname='', artid='', genre='', followers=-1, lessthan = True, llen = -1):
    if (artname == '' and artid == '' and genre == ''):
        print('please include name, id, or genre')
        return
    else:
        global query_count
        query_count = 0
        alist = []
        if (artname != ''):
            se = sp.search(q = artname, limit = 1, type = 'artist')
            query_count += 1
            print("related_artists search " + str(query_count))
            if (se['artists']['total'] > 0):
                artist = se['artists']['items'][0]
                artid = artist['id']
                genres = artist['genres']
        elif (artid != ''):
            artist = sp.artist(artid)
            genres = artist['genres']
            
#         print(artist)
        
        for a in sp.artist_related_artists(artid)['artists']:
            genres+=a['genres']
            
#         print(genres)

        lst = ra_help(artid, genres, followers, lessthan)
#         print(lst)
        query_count += 1
        print("related_artists ra_help " + str(query_count))
                
        if query_count < 10:
            lst+=(ra_rec(lst, genres, followers, lessthan))
          
        print(len(lst))
        
        print(len(sorted(list(set(lst)), key = custom_sort)))
    
    if llen > 0:
        return sorted(list(set(lst)), key = custom_sort, reverse = True)[0:llen]
    else:
        return sorted(list(set(lst)), key = custom_sort, reverse = True)


# related_artists(artname = 'berhana', followers = 75000)    

# Songkick Functions

[Back to top](#Table-of-Contents)

In [4]:
# function definitions

# takes json string from songkick for an artist and makes points based on the results
def make_points(json_str, artist):
    df = pd.DataFrame(
    {'Artist': [],
     'Venue': [],
     'City': [],
     'State/Province': [],
     'Country': [],
     'Longitude': [],
     'Latitude': []})

    i = 0
    
    if (json_str['resultsPage']['totalEntries'] != 0):
        for p in json_str['resultsPage']['results']['event']:
            #     print(p['popularity'])
            venue = p['venue']['displayName']
            lng = p['location']['lng']
            lat = p['location']['lat']
            city = p['location']['city']
            state = 'N/A'
            csplit = city.split(',')
            country = csplit[1]
        
            if (len(csplit) == 3):
                if not ('Washington' == csplit[0] and 'DC' == csplit[1]):
                    city = csplit[0]
                    state = csplit[1]
                    country = csplit[2]
                else:
                    city = csplit[0] + ', ' + csplit[1]
                    state = 'N/A'
                    country = csplit[2].replace(' ', '')
        
            df.loc[i] = [artist] + [venue] + [city] + [state] + [country] + [lat] +[lng]

            i+=1
        #convert to gdf to plot
        gdf = gpd.GeoDataFrame(
            df, geometry=[Point(x, y) for y, x in zip(df['Longitude'], df['Latitude'])])

        gdf2 = gdf
        gdf2 = gdf2.drop(list(range(1, len(gdf))))
        gdf3 = gdf
        gdf3 = gdf3.drop(list(range(0, len(gdf)-1)))
    
        return(gdf, gdf2, gdf3)
    else:
        return -1
    
def find_tour(json_str, artist, tr = 7, cutoff_yr = 2):
    df = pd.DataFrame(
    {'Artist': [],
     'Venue': [],
     'City': [],
     'State/Province': [],
     'Country': [],
     'Longitude': [],
     'Latitude': [],
     't': []})

    i = 0
    
    arr = []
    
    if (json_str['resultsPage']['totalEntries'] != 0):
        for p in json_str['resultsPage']['results']['event']:
            if 'US' in p['location']['city']:
                venue = p['venue']['displayName']
                lng = p['location']['lng']
                lat = p['location']['lat']
                city = p['location']['city']
                state = 'N/A'
                csplit = city.split(',')
                country = csplit[1]
        
                if (len(csplit) == 3):
                    if not ('Washington' == csplit[0] and 'DC' == csplit[1]):
                        city = csplit[0]
                        state = csplit[1]
                        country = csplit[2]
                    else:
                        city = csplit[0] + ', ' + csplit[1]
                        state = city
                        country = csplit[2].replace(' ', '')
                    df.loc[i] = [artist] + [venue] + [city] + [state] + [country] + [lat] + [lng] + [p['start']['date']]
            
                i+=1
            
                if (len(p['location']['city'].split(',')) == 3):
                    d = p['start']['date'].split('-')
                    arr.append(date(int(d[0]), int(d[1]), int(d[2])))
    else:
        return -1
    
    l = [0,0]
    first = 0
    last = 0
    for i in range(len(arr)-1):
        if (arr[i] != -1 and arr[i+1] != -1):
            if (((arr[i] - arr[i+1]).days > tr) or (i == len(arr)-2)):
                last = i
                if (last-first > l[1] - l[0]):
                    l = [first, last]
                first = i+1
        else:
            last = i
            if (last-first > l[1] - l[0]):
                l = [first, last]
            first = i+1
            
#     print(l)
    
    df2 = pd.DataFrame(
    {'Artist': [],
     'Venue': [],
     'City': [],
     'State/Province': [],
     'Country': [],
     'Longitude': [],
     'Latitude': [],
     't': []})
    
    if (len(df) > 0):
        for i in range(l[0], l[1]+1):
            df2.loc[i] = df.loc[i]
        
        #convert to gdf to plot
        gdf = gpd.GeoDataFrame(
                df2, geometry=[Point(x, y) for y, x in zip(df2['Longitude'], df2['Latitude'])])

        return gdf
    else:
        return -1

# plots singular tour path for artist
def run_plot(req, aname):
    gdfs = make_points(req.json(), aname)

    points_gdf = gdfs[0]
    print(points_gdf)
    start_gdf = gdfs[1]
    stop_gdf = gdfs[2]

    #current us basemap
    world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

    # fig, ax = plt.subplots(figsize=(15,15))
    ax = world[world.continent == 'North America'].plot(
#     ax = world.plot(
        color='white', edgecolor='black')

    ax.set_xlim(-130, -65)
    ax.set_ylim(24, 50)

    # We can now plot our ``GeoDataFrame``.
    points_gdf.plot(ax=ax, color='blue')

    start_gdf.plot(ax=ax, color='green')
    stop_gdf.plot(ax=ax, color='red')

    points = []

    linedf = pd.DataFrame(
        {'Line': []})

    for i in range(len(points_gdf)):
        points.append(points_gdf.iloc[i].loc["geometry"])

    line = LineString(points)

    def plot_line(ax, ob):
        x, y = ob.xy
        ax.plot(x, y, color='grey', alpha=0.7, linewidth=1, solid_capstyle='round', zorder=2)

    plot_line(ax, line)

    plt.tight_layout()

    plt.show()
    
# used to find the artist id on songkick
def get_artist_id(artist_name):
    json_str = requests.get("https://api.songkick.com/api/3.0/search/artists.json?apikey=" + api_key + "&query=" + artist_name).json()
    return (str(json_str['resultsPage']['results']['artist'][0]['id']) if json_str['resultsPage']['totalEntries'] != 0 else -1)

# function for interface for taking input
def req_input():
    print("Welcome")
    while (True):
        past = True
        is_venue = True
        
        cmd = input("Venue search (0) or event search (1)?")
        
        if (cmd == 1):
            is_venue = False
            
        if (is_venue == False):
            cmd = input("Past events (0), upcoming events (1): ")
        
            if (cmd == 1):
                past = False
            
            cmd = input("Please type the parameters: [artist_name, min_date, max_date.] Separate with commas."
                       + "\nIf no input, just add a comma and skip.\n")
        
            if (cmd == 'quit'):
                return
        
            cs = cmd.split(',')
            an = get_artist_id(cs[0].replace(' ', '+')) if not (cs[0]+' ').isspace() else print('need artist!')
            if (cs[0]+' ').isspace():
                continue
            mind = "&min_date=" + cs[1].replace(' ', '') if (re.search("\d{4}-\d{2}-\d{2}", cs[1])) else ''
            maxd = "&max_date=" + cs[2].replace(' ', '') if (re.search("\d{4}-\d{2}-\d{2}", cs[2])) else ''
            
            if (past == True):
                r = requests.get("https://api.songkick.com/api/3.0/artists/"+ an +"/gigography.json?apikey="+ api_key 
                              + mind + maxd)
                run_plot(r, cs[0])
            else:
                r = requests.get("https://api.songkick.com/api/3.0/events.json?apikey="+ api_key 
                    + an + loc + mind + maxd)
                print(r.json())
                run_plot(r, cs[0])
        else:
            cmd = input("Past venues (0), upcoming venues (1): ")
        
            if (cmd == 1):
                past = False
            
        cmd = input("Please type the parameters: [artist_name, min_date, max_date.] Separate with commas."
                   + "\nIf no input, just add a comma and skip.\n")
        
        if (cmd == 'quit'):
            return
        
        cs = cmd.split(',')
        an = get_artist_id(cs[0].replace(' ', '+')) if not (cs[0]+' ').isspace() else print('need artist!')
        if (cs[0]+' ').isspace():
            continue
        mind = "&min_date=" + cs[1].replace(' ', '') if (re.search("\d{4}-\d{2}-\d{2}", cs[1])) else ''
        maxd = "&max_date=" + cs[2].replace(' ', '') if (re.search("\d{4}-\d{2}-\d{2}", cs[2])) else ''
        
        if (past == True):
            r = requests.get("https://api.songkick.com/api/3.0/artists/"+ an +"/gigography.json?apikey="+ api_key 
                              + mind + maxd)
            run_plot(r, cs[0])
        else:
            r = requests.get("https://api.songkick.com/api/3.0/events.json?apikey="+ api_key 
                + an + loc + mind + maxd)
            print(r.json())
            run_plot(r, cs[0])
              

    
# req = requests.get("https://api.songkick.com/api/3.0/events.json?apikey=fCnCRQ1S6LDAS7zQ&artist_name=young+thug&min_date=2019-10-01&max_date=2019-11-01")

# req = requests.get("https://api.songkick.com/api/3.0/events.json?apikey=fCnCRQ1S6LDAS7zQ&artist_name=travis+scott&min_date=2018-01-01&max_date=2022-11-01")

# print(get_artist_id('travis+scott'))

# print("https://api.songkick.com/api/3.0/artists/"+ get_artist_id('travis+scott') + "/gigography.json?apikey=fCnCRQ1S6LDAS7zQ")

# print(req.text)
    

# Analysis Functions

[Back to top](#Table-of-Contents)

In [5]:
# this function generates many related artists, returns most recent events for each one, 
def compile_points(names = [], followers = -1, event_num=8):
    if not (names == []):
            print('yo')
            artists = []
            if followers == -1:   
                for n in names:
                    artists += related_artists(artname = n, llen = 10)
            else:
                for n in names:
                    artists += related_artists(artname = n, llen = 10, followers = followers)

#             print('llen')
#             print(artists)
            nl = []
            for a in artists:
                nl.append(a[0])
            
            nl = list(set(nl))
            
            gl = []
            
            for a in nl:
                an = get_artist_id(a.replace(' ', '+')) if not (a[0]+' ').isspace() else print('need artist!')
    
                if an != -1:
                    req = requests.get("https://api.songkick.com/api/3.0/artists/"+ an +"/gigography.json?apikey="+ api_key + '&per_page='+str(event_num)+'&order=desc')
#             print("https://api.songkick.com/api/3.0/artists/"+ an +"/gigography.json?apikey="+ api_key + '&per_page=20&order=desc')
                    gdfs = make_points(req.json(), a)
                    if gdfs != -1:
                        gl.append(gdfs[0])
                
            return gl
    else:
        return -1

# gdfs = compile_points(names = ['pat app', 'supakaine'], followers = 50000)
# gdf = compile_points(['pat app'])
# gdf2 = compile_points(['supakaine'])



In [6]:
def basic_stats(gdfs, state = ''):    
    rdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))
    
    # basic stats
    venues = {}
    cities = {}
    countries_states = {}
    
    for index, row in rdf.iterrows():
        if row[1] != 'Unknown venue':
            print(row)
            if row[1] not in venues:
                venues[row[1]] = 1
            else:
                venues[row[1]] += 1
          
        if row[3] != 'N/A':
            if (row[2] + ', ' + row[3][1:]) not in cities:
                cities[(row[2] + ', ' + row[3][1:])] = 1
            else:
                cities[(row[2] + ', ' + row[3][1:])] += 1
        else:
            if (row[2] + ', ' + row[4]) not in cities:
                cities[(row[2] + ', ' + row[4][1:])] = 1
            else:
                cities[(row[2] + ', ' + row[4][1:])] += 1
            
        if row[3] != 'N/A':
            if (row[3][1:]) not in countries_states:
                countries_states[row[3][1:]] = 1
            else:
                countries_states[row[3][1:]] += 1

        
        if (row[4][1:]) not in countries_states:
            countries_states[row[4][1:]] = 1
        else:
            countries_states[row[4][1:]] += 1
            
    print('Top 5 venues:')
    top = Counter(venues)
    for i in top.most_common(5):
        print(i[0] + ' ' + str(i[1]))
        
    print('\nTop 5 cities:')
    top = Counter(cities)
    for i in top.most_common(5):
        print(i[0] + ' ' + str(i[1]))
        
    print('\nTop 5 countries:')
    top = Counter(countries_states)
    for i in top.most_common(5):
        print(i[0] + ' ' + str(i[1]))
        
    return(venues)
        
    # end basic stats    

def autocorr(gdfs, state = ''):
    rdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))

    plt.figure(figsize=(10, 12))

    world = ws
    
    showcount = []
    
    for index, row in world.iterrows():
        if row[1] == 'United States':
            c = 'US'            
        elif row[1] == 'United Kingdom':
            c = 'UK'
        else:
            c = row[1]
#             print(c)
        
        if c in countries_states:
            showcount.append(countries_states[c])
        else:
            showcount.append(0)
        
    world["ShowCount"] = showcount #make new column

    for index, row in ws.iterrows():
        if row[1] == 'United States':
            world.drop(index, inplace=True)
    
    fig, ax = plt.subplots(figsize=(15,15)) #plot
    world.plot("ShowCount", ax=ax)
    plt.tight_layout()
    plt.show()
    
    # morans i

    qW = Queen.from_dataframe(world)
    
    qW.silent_island_warning = True

    qW.transform = 'r'

    #moran
    moran_sc = esda.moran.Moran(world[['ShowCount']], qW)

    #lisa
    lisa_sc = esda.Moran_Local(world["ShowCount"], qW)

    print(str(moran_sc.I) + ", " + str(moran_sc.p_sim))
    
    from pysal.viz.splot import esda as esdaplot
    esdaplot.plot_local_autocorrelation(lisa_sc, world, "ShowCount")

    plt.show()    
    
def analyze(gdfs, state = ''):
    rdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))
    basic_stats(gdfs)
    autocorr(gdfs)
    
def venue_count(gdfs, usonly = True):    
    rdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))
    
    venues = {}
    
    if (usonly == True):
        for index, row in rdf.iterrows():
            if row[1] != 'Unknown venue':
                if (row[4].replace(' ', '') == 'US'): 
                    if row[1] not in venues.keys():
                        venues[row[1]] = [(row[5],row[6], row[2], row[3].replace(' ', '')), 1]
                    else:
                        venues[row[1]][1] += 1
        
    return(venues)


def get_tracts(vens):
    tracts = {}
    
    for k,v in vens.items():
#         print(v)
        x = float(v[0][1])
        y = float(v[0][0])
        r = ''
        try:
            r = censusgeocode.coordinates(x=x, y=y)
        except:
            print('censusgeocode error')
            continue
        
        try:
            r = r['2010 Census Blocks']
            st = r[0]['STATE']
            co = r[0]['COUNTY']
            tr = r[0]['TRACT']
#             print(st + co + tr)
            b = cen.acs5.state_county_tract('B02001_002E,B02001_001E,B01002_001E,B19013_001E', st, co, tr)
            b2 = cen.acs5.state_county_tract('B02001_003E', st, co, tr)

            ftr = st + co + tr
            if (ftr not in tracts.keys()):
                tracts[ftr] = {'Non-white population percentage': -1,
                'Black population percentage': -1,
                'Median age': -1,
                'Median household income': -1,
#                 'Nonfamily households percentage': -1,
                'Count': -1}
                tracts[ftr]['Non-white population percentage'] = (1 - b[0]['B02001_002E']/b[0]['B02001_001E'])
                tracts[ftr]['Black population percentage'] = b2[0]['B02001_003E']/b[0]['B02001_001E']
                tracts[ftr]['Median age'] = b[0]['B01002_001E']
                tracts[ftr]['Median household income'] = b[0]['B19013_001E']
#                 tracts[ftr]['Nonfamily households percentage'] = b2[0]['B11001_007E']/b2[0]['B11001_001E']
                tracts[ftr]['Count'] = v[1]
            else:
#                 print(tracts)
                tracts[ftr]['Count'] += v[1]
        except KeyboardInterrupt:
            raise
        except:
            print('some other error')
            
        # b = c.acs5.state_place('B25034_010E', states.CA.fips, 'Los Angeles')
#         print(b)

    return tracts

        
# analyze(gdfs)

In [7]:
def tract_reg(tr, demo='non-white'):
#     print(tr)
    height = len(tr)
    width = len(next(iter(tr.values())))
    for k,vl in tr.items():
        v = list(vl.values())
        for i in range(width):
            if v[i] == -1:
#                 print(i)
                height = height - 1
                break

    print(height)
    print(width)

    arrr = [[0 for x in range(width-1)] for y in range(height)] 

    c = []

    i = 0
    for k,vl in tr.items():
        v = list(vl.values())

        br = False
        for j in range(width-1):
            if v[j] == -1:
#                 print(j)
                br = True

        if (br==True):
#             print('yo')
            continue

        c.append(v[width-1])
        for j in range(width-1):
            arrr[i][j] = v[j]
        i+=1
    
    if (demo == 'black'):
        for row in arrr:
            del row[0]
    else:
        for row in arrr:
            del row[1]
        
#     print(arrr)
    return statsmodels.regression.linear_model.OLS(c, arrr).fit().summary()


In [8]:
def tract_stats(names, followers, demo):
    points = compile_points(names, followers = followers)
    tracts = get_tracts(venue_count(points))
    print(tract_reg(tracts, demo=demo))
    
    
# tract_stats(names = ['pat app', 'supakaine'], followers = 50000, demo = 'black')

In [9]:
# def tract_stats(names, followers):

# tract_reg(tracts, demo='black')
    
    
# tract_stats(names = ['pat app', 'supakaine'], followers = 50000)

In [10]:
# # from statsmodels.stats.outliers_influence import variance_inflation_factor
# # print(arrr)
# print(len(c))
# statsmodels.regression.linear_model.OLS(c, arrr).fit().summary()
# # print(statsmodels.)
# X = numpy.array(arrr)
# # print(X)
# vif = [variance_inflation_factor(X, i) for i in range(X.shape[1])]
# print(vif)
# # statsmodels.regression.linear_model.OLS(c, arrr).fit().summary()


In [11]:
# req_input()

#travis scott, 2018-10-31, 2018-12-01

In [12]:
# se = sp.search(q = 'genre:%22southern%20hip%20hop%22', type = 'album', market = 'US')

#         for a in sp.artist_related_artists(artid)['artists']:

# this function generates many related artists, returns most recent events for each one, 
def compile_tours(names = [], followers = -1, lessthan = True):
    if (names != [] and isinstance(names, list)):
            print('yo')
            artists = []
            if followers == -1:   
                for n in names:
                    artists += related_artists(artname = n, llen = 10)
            else:
                for n in names:
                    artists += related_artists(artname = n, llen = 10, followers = followers)

#             print('llen')
            print(artists)
            nl = []
            for a in artists:
                nl.append(a[0])
            
            nl = list(set(nl))
            
            print(len(nl))
            
            gl = []
            
            for a in nl:
                an = get_artist_id(a.replace(' ', '+')) if not (a[0]+' ').isspace() else print('need artist!')
    
                if an != -1:
                    req = requests.get("https://api.songkick.com/api/3.0/artists/"+ an +"/gigography.json?apikey="+ api_key + '&per_page=50&order=desc')
#             print("https://api.songkick.com/api/3.0/artists/"+ an +"/gigography.json?apikey="+ api_key + '&per_page=20&order=desc')
                    g = find_tour(req.json(), a)
                    if g is not int:
                        gl.append(g)
            
            return gl
    else:
        return -1


In [13]:
def plot_all_tours(ts):
    m = folium.Map(location=[40, -95], zoom_start=4)

    colors = ['blue', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen']

    cc = -1

    for x in ts:
        if not isinstance(x, int) and len(x) != 0:
            if (cc!=len(colors)-1):
                cc += 1
            else:
                cc = 0
    
            points = []

            for i, row in x.iterrows():
                tooltip = 'Artist: '+ str(row[0]) + '- City: ' + str(row[2]) + ', ' + str(row[3])

                points.append((row[5], row[6]))

                folium.Circle([row[5], row[6]], radius = 8, color = colors[cc], fill = True, weight = 5, popup='<i></i>', tooltip=tooltip).add_to(m)

            folium.PolyLine(points, color=colors[cc], weight=2.5, opacity=1).add_to(m)
            
    return m


In [14]:
def optimal_path(ts):
    avg = 0
    maxm = 0
    cities = {}
    #     print(len(list))
    for t in ts:

        if not isinstance(t, int):
            for i,row in t.iterrows():
                city = str(row[2]) + ', ' + str(row[3]) + ', ' + str(row[4])
                if city not in cities.keys():
                    cities[city] = [(row[5], row[6]), 1]
                else:
                    cities[city][1] += 1

    lst = sorted(cities.items(), key = 
             lambda kv:(kv[1][1], kv[0]), reverse=True)

    maxm = lst[0][1][1]
    sl = []
    for i in lst:
        avg += i[1][1]
        sl.append(i[1][1])

    sd = statistics.stdev(sl)

    avg = avg/len(lst)

    green = []
    lg = []
    yellow = []
    red = []

    for i in lst:
        count = i[1][1]
        if (count >= avg + sd*1.5):
            green.append(i[1][0])
        elif (count >= avg):
            lg.append(i[1][0])
        elif (count >= avg - sd/2):
            yellow.append(i[1][0])
        else:
            red.append(i[1][0])

    final_points = green


    def dist(x,y):
        return math.hypot(y[0]-x[0],y[1]-x[1])

    paths = [ p for p in it.permutations(final_points) ]
    path_distances = [ sum(map(lambda x: dist(x[0],x[1]),zip(p[:-1],p[1:]))) for p in paths ]
    min_index = numpy.argmin(path_distances)

    print(str(paths[min_index]) + ' ' + str(path_distances[min_index]))

    final_path = []

    for i in paths[min_index]:
        final_path.append(i)

    print(final_path)

    def add_fp(fp, l, std):
        i = 0
        while (i < len(l)):
            if (l[i] not in fp):
                for j in range(len(fp)-1):
                    if ((dist(fp[j], l[i]) + dist(l[i], fp[j+1])) < (std*dist(fp[j], fp[j+1]))):
                        if (l[i] not in fp):
                            fp.insert(j+1, l[i])
                            print(l[i])
                            i = -1
                        continue
            i+=1

        return fp

    final_path = add_fp(final_path, lg, 1.5)
    final_path = add_fp(final_path, yellow, 1.2)
    final_path = add_fp(final_path, red, 1.05)

    print(final_path)

    m = folium.Map(location=[40, -95], zoom_start=4)

    count = 0
    for k, v in lst:
        color = ''
        if (v[0] in green):
            count+=1
            color = 'green'
        elif (v[0] in lg):
            count+=1
            color = 'lightgreen'
        elif (v[0] in yellow):
            count+=1
            color = 'yellow'
        elif (v[0] in red):
            count+=1
            color = 'red'
        else:
            print('oops')
                
        tooltip = 'City: ' + str(k) + ' - Count: ' + str(v[1])
        folium.Circle([v[0][0], v[0][1]], radius = 8, color = color, fill = True, weight = 5, popup='<i></i>', tooltip=tooltip).add_to(m)

    folium.PolyLine(final_path, color='blue', weight=1, opacity=1.5).add_to(m)
  
    return m
    
# optimal_path(ts)

# Analysis

[Back to top](#Table-of-Contents)

Hip-hop: Midwest, East, West, South

Each region uses four or five artists that are from that area and under that genre.

These artists are used as the root artists to find related artists to get a sample of performance data.

Under each genre/region pair, there is:
* a regression done on the tracts the venues are found in
* plot of each artist's longest tour path from their past 50 shows
* a plot of the optimal path an artist would take with color coded points to signify the quality of each stop
    * Green: Shows > average + 1.5*sd
    * light green: Shows > average
    * Yellow: Shows > average - sd/2
    * red: everything else 

## Hip-hop

x1: (Black population)/(Total Population)

x2: Median Age

x3: Median Household Income

### Midwest (https://djbooth.net/features/2015-11-03-next-up-in-the-midwest)

In [922]:
tract_stats(names = ['pat app', 'supakaine', 'Kweku Collins', 'IshDARR'], followers = 50000, demo = 'black')

yo
related_artists search 1
related_artists ra_help 3
56
28
related_artists search 1
related_artists ra_help 3
110
78
related_artists search 1
related_artists ra_help 3
65
18
related_artists search 1
related_artists ra_help 3
67
23
25017353101
some other error
censusgeocode error
39049003200
29095015800
08031003701
censusgeocode error
censusgeocode error
censusgeocode error
censusgeocode error
53033008600
17031330100
censusgeocode error
censusgeocode error
censusgeocode error
37183052602
censusgeocode error
censusgeocode error
36047044900
13121000500
censusgeocode error
censusgeocode error
06037700501
48113020400
censusgeocode error
censusgeocode error
13121000500
censusgeocode error
17031061100
09003500100
48113020400
06073002201
censusgeocode error
censusgeocode error
06083000900
06037208502
censusgeocode error
censusgeocode error
53033010300
08031001701
04013218300
censusgeocode error
13121011900
censusgeocode error
censusgeocode error
censusgeocode error
48201310100
censusgeocode e

In [16]:
tours = compile_tours(names = ['pat app', 'supakaine', 'Kweku Collins', 'IshDARR'], followers = 50000)

# plot_all_tours(tours)

yo
related_artists search 1
related_artists ra_help 3
54
28
related_artists search 1
related_artists ra_help 3
100
78
related_artists search 1
related_artists ra_help 3
72
19
related_artists search 1
related_artists ra_help 3
74
24
[('Tommy Ice', '3AUIYVmhwrsw8UOPHEv91Z', 1, 6454), ('Skrub', '4gBIvL912yEFl53RwEfLLV', 1, 3253), ('Nerve', '7BJleNy6CWKzS3nKtrsNuN', 1, 8071), ('Nter', '4DIp9TFBq2eh9sQbD5YT9I', 1, 12120), ('Gravy Baby', '1UXCGWYR853oxuWYI8hZWJ', 1, 4529), ('Cheap Sober', '6TKh8NPpURP4w3olT22ZsF', 1, 21322), ('Zotiyac', '26LmKYZRstUUMu2LC2xen9', 1, 9678), ('Huskii', '7uXoqYYwYwTUMF2zSF9Fwl', 1, 14433), ('Exl', '6j5lxsz5UsEHXTgkwVC4ie', 1, 1969), ('Lil Sknow', '6B2yDPPGFle5HGJKfKzhl3', 1, 10571), ('LNDN DRGS', '65d2Cw70rLuVwEZwQpHtFW', 3, 9910), ('Larry June', '1grN0519h2zYqpRtYbDZAl', 3, 31396), ('Vdon', '2uzaOMszD2lvSWPKjaYaTq', 2, 5223), ('Blade Icewood', '3WuFQUQAOAsFgnVlLKALM4', 2, 2569), ('Streetlordz', '3bo1dDhj5zc2dvlD9HfmEa', 2, 2306), ('Chris Crack', '3wws8s15DuuQWi

In [17]:
optimal_path(tours)

((42.33781, -83.05173), (41.9269089, -87.6488081), (32.78533, -96.80809), (39.74058, -104.97699)) 28.284840663169437
[(42.33781, -83.05173), (41.9269089, -87.6488081), (32.78533, -96.80809), (39.74058, -104.97699)]
(29.75393, -95.36406)
(30.26848, -97.73618)
(33.76423, -84.35023)
(29.42472, -98.49149)
(38.615345, -90.196469)
(39.06115, -94.59053)
(42.280221, -83.7512377)
(35.4935, -97.52016)
(38.9647957, -95.2358532)
(33.4971932, -86.7932486)
[(42.33781, -83.05173), (42.280221, -83.7512377), (41.9269089, -87.6488081), (38.615345, -90.196469), (33.76423, -84.35023), (33.4971932, -86.7932486), (39.06115, -94.59053), (38.9647957, -95.2358532), (35.4935, -97.52016), (29.42472, -98.49149), (30.26848, -97.73618), (29.75393, -95.36406), (32.78533, -96.80809), (39.74058, -104.97699)]


### West (https://djbooth.net/features/2017-08-22-west-coast-rappers-california-torch)

In [819]:
tract_stats(names = ['Dumbfoundead', 'Duckwrth', 'Trizz', 'Ill Camille', 'Chuuwee'], followers = 50000, demo = 'black')

yo
related_artists search 1
related_artists ra_help 3
79
41
related_artists search 1
related_artists ra_help 3
40
17
related_artists search 1
related_artists ra_help 3
69
42
related_artists search 1
related_artists ra_help 3
93
55
related_artists search 1
related_artists ra_help 3
76
42
25017353101
17031062800
censusgeocode error
29095015800
08031003701
49035101900
censusgeocode error
53033007500
06037207710
06073005300
06001402800
53033005301
08031002403
censusgeocode error
17031221500
09009361402
06067001200
06023011500
04013114100
censusgeocode error
censusgeocode error
19103002100
censusgeocode error
censusgeocode error
censusgeocode error
17031330100
48453000902
36047055700
11001003800
36047003300
36061019800
censusgeocode error
06075017802
censusgeocode error
36047044900
censusgeocode error
censusgeocode error
06037700501
06075020100
48113020400
48201310100
censusgeocode error
42101037600
51660000302
06073002201
04019000400
04005001200
06083000900
censusgeocode error
censusgeocod

In [18]:
tours = compile_tours(names = ['Dumbfoundead', 'Duckwrth', 'Trizz', 'Ill Camille', 'Chuuwee'], followers = 50000)

plot_all_tours(tours)

yo
related_artists search 1
related_artists ra_help 3
79
40
related_artists search 1
related_artists ra_help 3
42
18
related_artists search 1
related_artists ra_help 3
83
46
related_artists search 1
related_artists ra_help 3
93
56
related_artists search 1
related_artists ra_help 3
78
43
[('Ovan', '4FuRHFtCeoYaeeVDKyyvbS', 5, 18950), ('Crucial Star', '4vdAgNz4vrUZVvS0CaVvGJ', 4, 34182), ('Rad Museum', '6UdKtISMXYKSR8jcgtFePo', 3, 38156), ('The Quiett', '2qI1pO64eYqGUiv1XTw4cy', 3, 39518), ('HAON', '2krUNMgFZYm5s4Nn0g91W9', 2, 44478), ('Jvcki Wai', '4bjcB3ZKiHgPzJvY2S2FLN', 2, 23357), ('Verbal Jint', '24sQuJhQ85ZygDG7sUVUxR', 2, 41906), ('Nafla', '3Zn6C68VCosoQrxu4D2Btr', 2, 31685), ('DJ DOC', '3kDgt4vVKGTLmtC3FkOwWF', 2, 5179), ('NELL', '5WY88tCMFA6J6vqSN3MmDZ', 2, 31358), ('Elujay', '1CgbNAF3Stnz1Tpipu3xdO', 4, 31310), ('Childish Major', '4bMNzAopkqYTwkBLwMKsF1', 3, 44112), ('Zelooperz', '5j93hwFBNo29RJMsWvtzj8', 2, 10086), ('Pivot Gang', '0kDgxQlVYVecF6MCNWJTYE', 2, 25802), ('Dave B.'

In [19]:
optimal_path(tours)

((33.69984, -117.91945), (40.7688, -111.90283), (39.74058, -104.97699), (41.8882, -87.6292), (42.33781, -83.05173), (33.72493, -84.41448), (39.96609, -75.13663), (40.73959, -74.00736), (42.3637, -71.10131)) 62.96384286189236
[(33.69984, -117.91945), (40.7688, -111.90283), (39.74058, -104.97699), (41.8882, -87.6292), (42.33781, -83.05173), (33.72493, -84.41448), (39.96609, -75.13663), (40.73959, -74.00736), (42.3637, -71.10131)]
(38.9223987, -77.0422671)
(34.04494, -118.2655)
(39.98588, -83.00542)
(32.78533, -96.80809)
(30.26848, -97.73618)
(32.71636, -117.15961)
(33.44995, -112.07405)
(29.75393, -95.36406)
(42.34735, -71.09567)
(34.06194, -118.30884)
(34.09071, -118.38797)
(29.42472, -98.49149)
(40.72575, -73.9918)
(39.06115, -94.59053)
(38.70033, -90.37487)
(33.4577445, -111.9081615)
(40.72845, -74.00512)
(40.65, -73.95)
(38.99748, -77.02793)
(35.23942, -80.84584)
(42.26674, -71.80081)
(36.16097, -95.99464)
(39.2876066, -76.6068397)
(33.43689, -111.94373)
(41.31157, -72.92936)
(33.422

### East Coast (https://www.complex.com/music/2018/05/underground-rappers-you-should-know/dark-lo)

In [821]:
tract_stats(names = ['Mach-Hommy', 'Tha God Fahim', 'Crimeapple', 'Al Divino', 'Dark Lo'], followers = 50000, demo = 'black')

yo
related_artists search 1
related_artists ra_help 3
107
54
related_artists search 1
related_artists ra_help 3
130
38
related_artists search 1
related_artists ra_help 3
127
41
related_artists search 1
related_artists ra_help 3
0
0
related_artists search 1
related_artists ra_help 3
127
40
censusgeocode error
36055003300
25017353101
23005000100
censusgeocode error
50015953600
04013106600
06037700501
06037207710
censusgeocode error
06001402800
53033005301
08031002403
27053104400
censusgeocode error
09009361402
13089020900
06059075002
06019000300
06019000600
06019004408
12086002800
06037219901
22071007700
censusgeocode error
27123034201
26163522400
censusgeocode error
39049001820
36061012500
17031061100
27053106200
censusgeocode error
06037212900
42101037600
11001004801
censusgeocode error
42101014400
censusgeocode error
censusgeocode error
06075020100
censusgeocode error
censusgeocode error
48113020400
censusgeocode error
48201310100
13121000500
37119004900
09003500100
48113020400
census

In [20]:
tours = compile_tours(names = ['Mach-Hommy', 'Tha God Fahim', 'Crimeapple', 'Al Divino', 'Dark Lo'], followers = 50000)

# plot_all_tours(tours)

yo
related_artists search 1
related_artists ra_help 3
108
62
related_artists search 1
related_artists ra_help 3
130
34
related_artists search 1
related_artists ra_help 3
132
36
related_artists search 1
related_artists ra_help 3
0
0
related_artists search 1
related_artists ra_help 3
133
41
[('Planet Asia', '7dGfoIWAb0h9eGpl3vEkVo', 5, 28795), ('Westside Gunn', '0ABk515kENDyATUdpCKVfW', 4, 49307), ('Skyzoo', '0qHfqHyHZnxvv1lfkJraQc', 4, 38285), ('DJ Muggs', '7pDkjftAxiFHomWsPNkapS', 4, 30024), ('Termanology', '7oF48iRjggIDjtfRcmmMvl', 3, 23845), ('Homeboy Sandman', '5s1QeAf7U5ELP9PTeLaicb', 3, 27490), ('Reks', '6Apw0ReFBi9C1QBultw8pn', 3, 23848), ('Havoc', '6UnvX7Zx85VNjOpLdaq49W', 3, 46469), ('Zelooperz', '5j93hwFBNo29RJMsWvtzj8', 3, 10086), ('Armand Hammer', '3SCI7sXHUZIeDKMWC5NT9C', 3, 9996), ('Skyzoo', '0qHfqHyHZnxvv1lfkJraQc', 5, 38285), ('Westside Gunn', '0ABk515kENDyATUdpCKVfW', 5, 49307), ('Benny The Butcher', '5Matrg5du62bXwer29cU5T', 4, 37395), ('Roc Marciano', '5X0V59ipZnbvnub

In [21]:
optimal_path(tours)

((42.3635, -71.10162), (40.73959, -74.00736), (39.9589828, -75.1502637), (33.73967, -84.34497), (41.91041, -87.678), (44.96254, -93.2878), (33.69984, -117.91945)) 58.108508530622316
[(42.3635, -71.10162), (40.73959, -74.00736), (39.9589828, -75.1502637), (33.73967, -84.34497), (41.91041, -87.678), (44.96254, -93.2878), (33.69984, -117.91945)]
(37.7952, -122.228)
(47.6134145, -122.3443283)
(40.4287151, -79.9825489)
(41.30435, -72.92367)
(39.754434, -104.978614)
(40.71373, -73.93428)
(42.35211, -71.11942)
(44.94612, -93.0956)
(45.5229781, -122.6736892)
(34.047775, -118.3241134)
(34.04698, -118.2478)
(39.98588, -83.00542)
(42.38822, -71.10387)
(39.635458, -106.537501)
(40.76683, -111.90889)
(40.73177, -73.98911)
(40.7284492, -74.0051241)
(43.0797373, -89.3764856)
(34.10912, -118.19457)
(40.26791, -76.88848)
(40.5852563, -105.0769611)
(33.74096, -84.34593)
[(42.3635, -71.10162), (42.35211, -71.11942), (42.38822, -71.10387), (41.30435, -72.92367), (40.71373, -73.93428), (40.73177, -73.98911

### South (https://www.complex.com/music/2018/05/underground-rappers-you-should-know/dark-lo)

In [825]:
tract_stats(names = ['D Flowers', 'AJ McQueen', 'Lajan Slim', 'Stunna 4 Vegas', 'Deniro Farrar'], followers = 50000, demo = 'black')

yo
related_artists search 1
related_artists ra_help 3
0
0
related_artists search 1
related_artists ra_help 3
121
75
related_artists search 1
related_artists ra_help 3
105
75
related_artists search 1
related_artists ra_help 3
42
33
related_artists search 1
related_artists ra_help 3
66
47
36081055900
42101038300
25017353101
25017353101
53033006600
13121010304
12057003600
12095018900
12099007759
censusgeocode error
13121006500
12086000501
48201230200
48113020400
censusgeocode error
12086002004
06037207302
51019030103
06073019303
censusgeocode error
12086002300
06037212900
06037700501
48201310100
06037195803
11001004801
08001009308
32003001200
04013421400
censusgeocode error
35001002100
06037206050
04013421400
13121003100
13121006500
censusgeocode error
censusgeocode error
13245010104
36047022100
12057010213
39017014000
censusgeocode error
39165031904
censusgeocode error
39017010909
39061023100
13121009700
12031002502
censusgeocode error
06075020700
06073005200
04013061020
32003003411
5154

In [32]:
tours = compile_tours(names = ['D Flowers', 'AJ McQueen', 'Lajan Slim', 'Stunna 4 Vegas', 'Deniro Farrar'], followers = 50000)

plot_all_tours(tours)

yo
related_artists search 1
related_artists ra_help 3
0
0
related_artists search 1
related_artists ra_help 3
121
74
related_artists search 1
related_artists ra_help 3
107
77
related_artists search 1
related_artists ra_help 3
47
38
related_artists search 1
related_artists ra_help 3
66
46
[("Thi'sl", '1Q45nhioxaxZTDjR0iaMWp', 3, 37889), ('Reconcile', '205ryRzLS0HTtASNep0wUm', 3, 11892), ('Fedel', '7EWlStyJsdfPvz0xeGBgTy', 3, 7682), ('Mouthpi3ce', '5ccLg9PXtgNpXdkIzw7Pvw', 3, 3473), ('K-Drama', '1PXBwNCd6LaX8iJTDwdC30', 3, 10361), ('R-Swift', '7gZ8kNeMcOD0Fm4b9xyOLX', 3, 9517), ('Kid Tris', '2GxwBfxo19utaZfuCUkTcv', 2, 4258), ('Corey Paul', '25ksFneQI7WRWIerxd8eg0', 2, 7762), ('Sho Baraka', '3dpswct31rsudWf8ZASWZe', 2, 39439), ("Torey D'Shaun", '78DvQP3rczGqfgEiLfFnCD', 2, 4776), ('Derin Falana', '5yjfk1YZOcpDKqLEgvE9WG', 4, 10136), ('ShaqIsDope', '3Is1kGaN2jmS0hB1Hn6dp4', 4, 22742), ('IshDARR', '1MlV4b7HGi4AiZWXholyVu', 3, 46249), ('Zoey Dollaz', '39Uhf21aJFjMLus1j0YGT8', 3, 36500), ('De

In [33]:
optimal_path(tours)

((42.363697, -71.1013086), (40.7224278, -73.9886921), (39.96609, -75.13663), (42.33781, -83.05173), (41.8882, -87.6292), (33.73649, -84.41038), (29.7604267, -95.3698028), (29.42472, -98.49149), (32.857314, -96.600582), (47.58791, -122.33401)) 74.69079297247688
[(42.363697, -71.1013086), (40.7224278, -73.9886921), (39.96609, -75.13663), (42.33781, -83.05173), (41.8882, -87.6292), (33.73649, -84.41038), (29.7604267, -95.3698028), (29.42472, -98.49149), (32.857314, -96.600582), (47.58791, -122.33401)]
(36.8488298, -75.975449)
(38.5815719, -121.4943996)
(45.52274, -122.6848)
(39.7392358, -104.990251)
(40.7143, -73.95587)
(30.2670974, -97.7363767)
(34.0815247, -118.3891802)
(38.9223987, -77.0422671)
(32.22494, -110.96898)
(36.97136, -122.0256)
(35.81684, -78.62071)
(33.5891372, -112.1514437)
(37.7952, -122.228)
(35.1497562, -90.014279)
(34.06194, -118.30884)
(36.1699412, -115.1398296)
(30.3265124, -81.6566148)
(28.5412984, -81.380945)
(27.966052, -82.410312)
(27.77167, -82.63604)
(25.92508,

## Rock (incomplete)

### West

(https://www.pastemagazine.com/blogs/lists/2014/07/10-nevada-bands-you-should-listen-to-now.html)
(https://buzzbands.la/2017/12/27/bands-watch-2018-20-artists-new-year/)
(https://www.pastemagazine.com/blogs/lists/2012/05/1x-washington-bands-you-should-listen-to-now.html)

In [827]:
tract_stats(names = ['The Bonfire Set', 'Elephant Rifle', 'LO MOON', 'PINKY PINKY', 'My Goodness'], followers = 50000, demo='non-white')


yo
related_artists search 1
related_artists ra_help 3
0
0
related_artists search 1
related_artists ra_help 3
128
72
related_artists search 1
related_artists ra_help 3
93
46
related_artists search 1
related_artists ra_help 3
0
0
related_artists search 1
related_artists ra_help 3
96
51
47037016200
26163507500
18097355900
21009950200
25017339801
censusgeocode error
36067003200
55073000100
42071000100
censusgeocode error
censusgeocode error
24510120600
42101037600
censusgeocode error
53033006600
21111006500
39093060100
censusgeocode error
42003411000
47031970500
47141000600
47035970602
06037265301
06037190201
censusgeocode error
censusgeocode error
25009217400
08031002403
08069000100
22071007800
censusgeocode error
06073017200
06037190510
06037215101
06075016200
49035114000
08031001600
29095015800
38017000700
27137001900
27123032100
27103480600
27099000410
censusgeocode error
24510120500
42101036900
39049000600
censusgeocode error
27123034201
53033004700
41051002203
18105000100
44005040900

In [931]:
tours = compile_tours(names = ['The Bonfire Set', 'Elephant Rifle', 'LO MOON', 'PINKY PINKY', 'My Goodness'], followers = 50000)

plot_all_tours(tours)

yo
related_artists search 1
related_artists ra_help 3
0
0
related_artists search 1
related_artists ra_help 3
128
72
related_artists search 1
related_artists ra_help 3
93
46
related_artists search 1
related_artists ra_help 3
0
0
related_artists search 1
related_artists ra_help 3
96
51
[('The Jesus Lizard', '6r26MaDr8bqNALjXgYPXMa', 14, 46874), ('Big Black', '5EYkvHZuGM3pwU3DZUrrZ3', 14, 41196), ('Wipers', '0sTTw3dw3EA0c7NaZnrJd2', 14, 28886), ('Mission Of Burma', '5UMQV83wwZCCvRSQCVjZw6', 13, 37738), ('Unwound', '4YjpqCSDD7zwMQgPYJMqb0', 13, 28694), ('Shellac', '6I8R5MFTlez7rHCsH4cx0u', 12, 48090), ('Brainiac', '0No2Og1pAwgw0eMmY6PPVP', 9, 8955), ('Lightning Bolt', '2og3FOCLYXT9H7IYE6QPUq', 9, 44470), ('Flipper', '20VDXslLsXlLgSF765m1Ug', 9, 22679), ('Scratch Acid', '219VCiRViG60EbPiocSWJp', 8, 16437), ('Talos', '5pdzKTGQAcRcxDOfN4mXSc', 7, 38802), ('VHS Collection', '2Nvaq4y2ygxIqfwXyz0HeH', 7, 43617), ('Mating Ritual', '4ZeB1hzT2mSZrf7wszOqHs', 6, 19291), ('courtship.', '2OK16hAFRHoJi

In [10]:
optimal_path(tours)

NameError: name 'optimal_path' is not defined

### East

(https://gooddeedseats.com/45-up-and-coming-rock-bands/)

In [834]:
tract_stats(names = ['Air Traffic Controller', 'Big Thief', 'J and The 9s', 'Charly Bliss', 'Hop Along'], followers = 50000, demo='non-white')


yo
related_artists search 1
related_artists ra_help 3
61
40
related_artists search 1
related_artists ra_help 3
75
24
related_artists search 1
related_artists ra_help 3
42
23
related_artists search 1
related_artists ra_help 3
130
50
related_artists search 1
related_artists ra_help 3
126
78
42071000100
51660000402
37021000100
51760040800
34029723600
24035810800
42071000100
42071000100
06075010600
06083000900
06019002200
06079011101
06067001400
06087101000
06073005300
06037207400
06037183810
06037191710
27123032100
29095016700
08031001600
17031062800
05119004400
47037016200
censusgeocode error
18097355900
censusgeocode error
censusgeocode error
48309000400
censusgeocode error
17143002800
17031061100
13121000500
49035114000
49035114000
49011126001
49035114000
49049002400
49049002400
censusgeocode error
06073002201
06037194402
06059088602
censusgeocode error
censusgeocode error
06075022702
17031070300
26081002000
50007000800
25017351100
09009165400
34017006500
47037019500
censusgeocode erro

In [ ]:
tours = compile_tours(names = ['Air Traffic Controller', 'Big Thief', 'J and The 9s', 'Charly Bliss', 'Hop Along'], followers = 50000)

plot_all_tours(tours)

In [ ]:
optimal_path(tours)

## Midwest

(https://gooddeedseats.com/45-up-and-coming-rock-bands/)
(https://radiomilwaukee.org/discover-music/tapd-in/wisconsin-bands-to-watch-in-2019/)

In [846]:
tract_stats(names = ['Hemmingbirds', 'The Accidentals', 'The Cactus Blossoms', 'Chicken Wire Empire', 'Rodeo Ruby Love'], followers = 50000, demo='non-white')


yo
related_artists search 1
related_artists ra_help 3
1
1
related_artists search 1
related_artists ra_help 3
105
55
related_artists search 1
related_artists ra_help 3
108
64
related_artists search 1
related_artists ra_help 3
118
48
related_artists search 1
related_artists ra_help 3
103
41
34023007103
36029016500
25025000704
17031081600
34001001300
06037208502
06073002201
04013113100
04005001200
08041006700
08031001600
08069000100
49035114000
36047049900
17031051100
06075022702
06037181000
36047055500
25027731700
06001431000
48091310503
48201411000
48113004600
56001963400
censusgeocode error
17031310600
39035107701
51760030500
42101036900
11001004400
17031809900
26161400100
45045002504
censusgeocode error
17019011100
27169670400
55123960400
19103002100
25017353000
25017353101
25013813207
25013810901
08067971100
55087010100
16001000100
41029002000
censusgeocode error
53063003500
30067000200
censusgeocode error
55103970100
06001402800
06075016400
censusgeocode error
51760030200
censusgeoc

In [ ]:
tours = compile_tours(names = ['Hemmingbirds', 'The Accidentals', 'The Cactus Blossoms', 'Chicken Wire Empire', 'Rodeo Ruby Love'], followers = 50000)

plot_all_tours(tours)

In [ ]:
optimal_path(tours)

## South

(https://gooddeedseats.com/45-up-and-coming-rock-bands/)
(https://www.pastemagazine.com/blogs/lists/2013/08/12-virginia-bands-you-should-listen-to-now.html)
(https://clture.org/10-carolina-bands/)

In [847]:
tract_stats(names = ['Savoy Motel', 'Ringo Deathstarr', 'Gull', 'Shadowgraphs', 'Young Mister'], followers = 50000, demo='non-white')


yo
related_artists search 1
related_artists ra_help 3
109
74
related_artists search 1
related_artists ra_help 3
119
30
related_artists search 1
related_artists ra_help 3
0
0
related_artists search 1
related_artists ra_help 3
108
71
related_artists search 1
related_artists ra_help 3
93
49
41051001101
06001983200
41051003402
41051001301
41051004200
53033009200
41051002100
53067010520
05143011300
censusgeocode error
39061001000
36029016500
censusgeocode error
censusgeocode error
42101014200
53033004700
41051005100
censusgeocode error
censusgeocode error
06059074106
censusgeocode error
censusgeocode error
26081002000
censusgeocode error
26125173500
06037209300
censusgeocode error
53033010300
censusgeocode error
censusgeocode error
17031833000
censusgeocode error
16001000100
censusgeocode error
censusgeocode error
48453001100
22071011100
29095016800
27053006800
55025001900
censusgeocode error
36047057900
42101002400
34025807003
47037019500
47037019500
37119001400
37135010705
censusgeocode e

In [ ]:
tours = compile_tours(names = ['Savoy Motel', 'Ringo Deathstarr', 'Gull', 'Shadowgraphs', 'Young Mister'], followers = 50000)

plot_all_tours(tours)

In [ ]:
optimal_path(tours)